### connecting our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

##### note: To run SQL queries in jupyter notebook, we add %%sql on its own line to the start of our query

### querying the database to get a list of all tables and views in our database:

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


#### schema diagram

![Chinook schema diagram](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

#### THE TASK 1


The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

    *Artist Name*	             *Genre*

    Regal	                Hip-Hop

    Red Tone	             Punk

    Meteor and the Girls	Pop

    Slim Jim Bites	       Blues


The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

### SOLUTION 

In [3]:
%%sql
WITH 
    
    bill_coun as (SELECT * FROM invoice WHERE billing_country="USA" ),
        
    inv_line as (SELECT il.*,bill_coun.*  FROM bill_coun
                  LEFT JOIN invoice_line il ON il.invoice_id=bill_coun.invoice_id),
    
    t_table as (SELECT t.*,inv_line.* FROM inv_line LEFT JOIN track t ON t.track_id=inv_line.track_id ),
    
    complete_table_genre as (SELECT t_table.*,g.name as genre FROM t_table LEFT JOIN genre g ON g.genre_id=t_table.genre_id)
     
SELECT genre,COUNT(genre) "number of tracks sold" , ROUND(Cast(Count(genre)as float)/(SELECT COUNT(genre) FROM complete_table_genre),2)*100 as "tracks_sold(%)",ROUND(SUM(unit_price),2) "total sales gain" ,  billing_country FROM complete_table_genre  GROUP BY 1 ORDER BY 2 DESC


 * sqlite:///chinook.db
Done.


genre,number of tracks sold,tracks_sold(%),total sales gain,billing_country
Rock,561,53.0,555.39,USA
Alternative & Punk,130,12.0,128.7,USA
Metal,124,12.0,122.76,USA
R&B/Soul,53,5.0,52.47,USA
Blues,36,3.0,35.64,USA
Alternative,35,3.0,34.65,USA
Pop,22,2.0,21.78,USA
Latin,22,2.0,21.78,USA
Hip Hop/Rap,20,2.0,19.8,USA
Jazz,14,1.0,13.86,USA


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

1) Red Tone (Punk)

2) Slim Jim Bites (Blues)

3) Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up
only **17% of the total tracks sold** , so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for **53% of the total tracks sold.**

In [4]:
%%sql
SELECT COUNT(name),COUNT(genre_id) FROM genre

 * sqlite:///chinook.db
Done.


COUNT(name),COUNT(genre_id)
25,25


In [5]:
%%sql
SELECT COUNT(track_id),COUNT( DISTINCT genre_id) FROM track

 * sqlite:///chinook.db
Done.


COUNT(track_id),COUNT( DISTINCT genre_id)
3503,25


### TASK 2

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
%%sql

WITH 

join_table AS (SELECT c.customer_id,c.support_rep_id,e.*,i.total FROM customer c
               LEFT JOIN employee e ON c.support_rep_id=e.employee_id
              LEFT JOIN invoice i ON i.customer_id=c.customer_id)


SELECT employee_id,first_name||" "||last_name "employee name",birthdate,hire_date,country,ROUND(sum(total),2) as "total sales generated", ROUND((Sum(total)/(SELECT SUM(total) FROM join_table))*100,2) "total_sales_generated(%)",count(customer_id) as "number of sales transactions supported" , COUNT(DISTINCT customer_id) "number of assigned customers", ROUND((CAST(COUNT(DISTINCT customer_id) as FLOAT)/(SELECT COUNT( DISTINCT customer_id) FROM join_table))*100,2) as "number of assigned customers(%)" FROM join_table
GROUP BY 1
ORDER BY 6 DESC

 * sqlite:///chinook.db
Done.


employee_id,employee name,birthdate,hire_date,country,total sales generated,total_sales_generated(%),number of sales transactions supported,number of assigned customers,number of assigned customers(%)
3,Jane Peacock,1973-08-29 00:00:00,2017-04-01 00:00:00,Canada,1731.51,36.77,212,21,35.59
4,Margaret Park,1947-09-19 00:00:00,2017-05-03 00:00:00,Canada,1584.0,33.63,214,20,33.9
5,Steve Johnson,1965-03-03 00:00:00,2017-10-17 00:00:00,Canada,1393.92,29.6,188,18,30.51


There are 3 sales support agents among the employees, they are all from Canada and were employed in 2017 (April, May and October). The youngest of them is Jane Peacock, the oldest - Margaret Park.

On one hand, we see a direct correlation between the hire date of a sales support agent and the total amount of sales. However, we also see that the first 2 employees have only 1 month difference in their hire date; moreover, Margaret Park, being hired later, had 2 more number of sales transactions supported than Jane who was hired early on . Despite that, they have approximately 3% difference in their total amounts of sales generated, whereas Steve Johnson, being hired more than 6 months after Jane Peacock, has only 7% less sales generated than Jane, and 4% less sales generated than Margaret .

Given that, Jane Peacock demonstrates the best sales performance, while Margaret Park - the worst.

In [7]:
%%sql
SELECT DISTINCT support_rep_id FROM customer

 * sqlite:///chinook.db
Done.


support_rep_id
3
4
5


In [8]:
%%sql
SELECT count(customer_id) FROM customer

 * sqlite:///chinook.db
Done.


count(customer_id)
59


### TASK 3

Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

1) total number of customers

2) total value of sales

3) average value of sales per customer

4) average order value



Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis

In [55]:
%%sql
WITH c_table as (SELECT * FROM customer ),

    i_table as (SELECT * FROM c_table
                  LEFT JOIN  invoice
                  ON c_table.customer_id=invoice.customer_id),
    group_table as (SELECT country,COUNT(DISTINCT customer_id) total_num_of_customers,ROUND(SUM(total),2) total_value_of_sales,ROUND(CAST(SUM(total) AS FLOAT)/COUNT(DISTINCT customer_id),2) average_value_of_sales_per_customer,COUNT(invoice_id) number_of_orders,COUNT(invoice_id)/COUNT(DISTINCT customer_id) "average_order_value(avg_num_of_orders)",CASE
                   WHEN COUNT(DISTINCT customer_id)=1 THEN "OTHER"
                   ELSE country
                   END as catergory_country 
                
                FROM i_table
               GROUP BY 1
               ORDER BY 2 DESC),
    
                  
    final_group_table as (SELECT catergory_country as country,sum(total_num_of_customers) as total_num_of_customers ,sum(total_value_of_sales) as total_value_of_sales ,sum(total_value_of_sales)/sum(total_num_of_customers) as average_value_of_sales_per_customer,sum(number_of_orders) as "num_orders",sum(number_of_orders)/sum(total_num_of_customers) "avg_num_of_orders"
                          FROM group_table
                          GROUP BY 1
                          ORDER BY 2 DESC)

SELECT country, total_num_of_customers ,total_value_of_sales,ROUND(average_value_of_sales_per_customer,2),num_orders,avg_num_of_orders
FROM (SELECT
        final_group_table.*,
        CASE
            WHEN final_group_table.country = "OTHER" THEN 1
            ELSE 0
        END AS sort
    FROM final_group_table)
ORDER BY sort ASC,total_value_of_sales DESC

                  

    


 * sqlite:///chinook.db
Done.


country,total_num_of_customers,total_value_of_sales,"ROUND(average_value_of_sales_per_customer,2)",num_orders,avg_num_of_orders
USA,13,1040.49,80.04,131,10
Canada,8,535.59,66.95,76,9
Brazil,5,427.68,85.54,61,12
France,5,389.07,77.81,50,10
Germany,4,334.62,83.66,41,10
Czech Republic,2,273.24,136.62,30,15
United Kingdom,3,245.52,81.84,28,9
Portugal,2,185.13,92.57,29,14
India,2,183.15,91.58,21,10
OTHER,15,1094.94,73.0,147,9


The sales by country are evidently dominated by the USA, in terms of both the total sales and the number of customers (if not to count the "Other" group). Besides, we can notice a strong positive correlation between the number of customers and the total sales. Interesting that Czech Republic, having only 2 customers, shows the highest average value of sales per customer and, consequently, the highest average order value

#### TASK 4

Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:

Number of invoices

Percentage of invoices

##### BACKGROUND STORY
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

1) purchase a whole album

2) purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

1) Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.

2) Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.


In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.



In [57]:
%%sql
SELECT* FROM INVOICE

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
6,31,2017-01-10 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98
7,49,2017-01-12 00:00:00,Ordynacka 10,Warsaw,None,Poland,00-358,10.89
8,59,2017-01-13 00:00:00,"3,Raj Bhavan Road",Bangalore,None,India,560001,9.9
9,18,2017-01-18 00:00:00,627 Broadway,New York,NY,USA,10012-2612,8.91
10,31,2017-01-18 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98


In [59]:
%%sql
SELECT* FROM INVOICE_line

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


every customer has an invoice which summarizes all the purchases made by a customer as a lump sum. an invoice line gives a breakdown of the details of each purchase within an invoice. ie sereval invoice line ids can correlate to a single invoice id



In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.



#### option 1

croseecheck using the **COUNT aggregate function**

1) the album_id associated with the tracks for a particular invoice is distinct/only one,(if so then there’s a probability is an album purchase)
note: if the album id is not distinct for an invoice automatically it cannot be an album purchase

2) even if point one is true , we have to confirm if all the tracks within the distinct album are indeed represented for that particular invoice_id.

if the two conditions hold then it can be classified as an album purchase hence it was a single purchase


In [139]:
%%sql

WITH invoice_d AS (SELECT * FROM invoice_line il LEFT JOIN track t ON t.track_id=il.track_id ),

    album_d AS (select count (track_id) tracks_in_album, album_id from track group by 2),
    
    final_table AS (SELECT  ind.invoice_id,count(ind.track_id) count_track_id_in_invoice,MIN(ind.track_id) as rep_track_id,count(distinct ind.album_id) albums__id_in_invoice_id,ind.album_id as rep_album_id ,ad.tracks_in_album from invoice_d ind  LEFT JOIN album_d ad ON ad.album_id=ind.album_id group by 1),

    catergorised_table AS (SELECT ft.*, CASE WHEN (ft. albums__id_in_invoice_id =1) AND (ft.count_track_id_in_invoice=ft.tracks_in_album) THEN "album" ELSE "single" END as type_of_purchase FROM final_table ft)
    
    
    SELECT type_of_purchase, count(invoice_id) number_of_invoices,ROUND((CAST(count(invoice_id)AS FLOAT)/(SELECT COUNT(invoice_id) FROM catergorised_table))*100,2) as "percentage invoice"  FROM catergorised_table ct GROUP BY 1

 * sqlite:///chinook.db
Done.


type_of_purchase,number_of_invoices,percentage invoice
album,114,18.57
single,500,81.43


Hence, customers tend to buy individual tracks 4 times more often than the whole albums. This means that the new strategy in consideration (purchasing only the most popular tracks from each album from record companies instead of purchasing the whole albums) will definitely help to maximize the overall revenue.

#### option 2

focused on comparing the contents using the **except clause**(a set of tracks ids within a specific invoice_id to a set of track ids within the album_id associated to the same invoice_id)


for each invoice_id select a represntative track_id and find its corresponding album_id to be a representative album_id for the invoice(nb.for invoice to be an album purcahse all the tracks_id associated to the invoice_id must have a distinct album_id/only one album_id and also the tracks ids should be the same as what is contained in the original distinct album_id)




In [141]:
%%sql
WITH invoice_d AS (SELECT * FROM invoice_line il LEFT JOIN track t ON t.track_id=il.track_id ),

    album_d AS (select count (track_id) tracks_in_album, album_id from track group by 2),
    
    final_table AS (SELECT  ind.invoice_id,count(ind.track_id) count_track_id_in_invoice,MIN(ind.track_id) as rep_track_id,count(distinct ind.album_id) albums__id_in_invoice_id,ind.album_id as rep_album_id ,ad.tracks_in_album from invoice_d ind  LEFT JOIN album_d ad ON ad.album_id=ind.album_id group by 1)

    
SELECT  count(invoice_id) "number of invoices",ROUND((CAST(count(invoice_id)AS FLOAT)/(SELECT COUNT(invoice_id) FROM final_table))*100,2) "invoice in %", 

CASE WHEN

(SELECT il.track_id FROM invoice_line il WHERE il.invoice_id=ft.invoice_id EXCEPT SELECT t.track_id FROM track t WHERE t.album_id=ft.rep_album_id)IS NULL

and 

(SELECT t.track_id FROM track t WHERE t.album_id=ft.rep_album_id EXCEPT SELECT il.track_id FROM invoice_line il WHERE il.invoice_id=ft.invoice_id)IS NULL 

THEN "ALBUM"

ELSE "SINGLE" 

END AS purchase_type

FROM final_table ft

GROUP BY purchase_type

 * sqlite:///chinook.db
Done.


number of invoices,invoice in %,purchase_type
114,18.57,ALBUM
500,81.43,SINGLE


### TASK 5
How many tracks have been purchased vs not purchased?

In [149]:
%%sql

SELECT count(distinct il.track_id) FROM INVOICE_LINE il

 * sqlite:///chinook.db
Done.


count(distinct il.track_id)
1806


In [150]:
%%sql

SELECT count(t.track_id) FROM track t

 * sqlite:///chinook.db
Done.


count(t.track_id)
3503


In [167]:
%%sql

WITH FINAL_TABLE AS 

(SELECT t.track_id,
       il.track_id as sold_track_id,
       CASE WHEN t.track_id = il.track_id  THEN "PURCHASED" ELSE "NOT PURCHASED" END AS "SOLD OR NOT"
        
        FROM track t
        LEFT JOIN invoice_line il ON t.track_id=il.track_id 
        GROUP BY 1)

SELECT count(track_id)-count(sold_track_id) as "number of tracks not purchased",count(sold_track_id) as "number of tracks sold",count(track_id) as "total number of tracks",ROUND((CAST(count(sold_track_id) as FLOAT)/count(track_id))*100,0) pct_sold FROM FINAL_TABLE

 * sqlite:///chinook.db
Done.


number of tracks not purchased,number of tracks sold,total number of tracks,pct_sold
1697,1806,3503,52.0


Almost half of all the unique tracks available in the Chinook store were never bought, probably being of unpopular genre or unpopular artists. Possible solutions here may include a new advertising campaign (especialy in the USA, as the biggest segment of market), sales promotion, additional training and motivation of the sales support agents.

### TASK 6
Which artist is used in the most playlists?

Next, we're going to find out which artists are used in the most playlists. This information can give us valuable insights on further purchasing strategies of the Chinook store. The tables to be used are artist, album, track, and playlist_track.

In [198]:
%%sql

SELECT ar.name,COUNT(DISTINCT pt.playlist_id),g.*

FROM playlist_track pt 

LEFT JOIN Track t on t.track_id=pt.track_id 

LEFT JOIN album al ON al.album_id=t.album_id 

LEFT JOIN artist ar ON ar.artist_id=al.artist_id 

LEFT JOIN genre g ON g.genre_id =t.genre_id

GROUP BY  1

ORDER BY 2 DESC



 * sqlite:///chinook.db
Done.


name,COUNT(DISTINCT pt.playlist_id),genre_id,name_1
Eugene Ormandy,7,24,Classical
The King's Singers,6,24,Classical
English Concert & Trevor Pinnock,6,24,Classical
Berliner Philharmoniker & Herbert Von Karajan,6,24,Classical
Academy of St. Martin in the Fields & Sir Neville Marriner,6,24,Classical
Yo-Yo Ma,5,24,Classical
Wilhelm Kempff,5,24,Classical
Ton Koopman,5,24,Classical
"Sir Georg Solti, Sumi Jo & Wiener Philharmoniker",5,25,Opera
Sir Georg Solti & Wiener Philharmoniker,5,24,Classical


We got a table of the TOP5 artists, with Eugene Ormandy at the first place (present in 7 playlists), and followed by 4 other artists (present in 6 playlists). the genre associated with the top 5 artists are related is Classical.

From one of our previous queries (calculating the number and \% of tracks of each genre sold in the USA), we remember that Classical genre, with all due respect, is far away from being the most sold one. Hence, in spite of these artists being present in the most playlists, a strategy for the Chinook store to buy more tracks of theirs cannot be considered as potentially profitable and should be disregarded.

### TASK 7

Do protected vs non-protected media types have an effect on popularity?

felt popularity means it was popular hence purchased

In [185]:
%%sql

SELECT* FROM media_type

 * sqlite:///chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [201]:
%%sql

WITH final_table as (SELECT il.invoice_line_id,il.track_id,t.media_type_id,mt.name,

                     CASE WHEN mt.name LIKE "%protected%" THEN "PROTECTED" ELSE "NON PROTECTED" END AS media_catergory

                    FROM invoice_line il
                     
                    LEFT JOIN track t ON t.track_id=il.track_id
                     
                    LEFT JOIN media_type mt ON mt.media_type_id=t.media_type_id)
      

SELECT media_catergory,count(distinct(track_id)) "number of unique sold tracks type", count(media_catergory) "number of types sold(overall)",ROUND((CAST(count(media_catergory) AS FLOAT)/(SELECT count(media_catergory) FROM final_table))*100,2)as "%number of types" FROM final_table GROUP BY 1

 * sqlite:///chinook.db
Done.


media_catergory,number of unique sold tracks type,number of types sold(overall),%number of types
NON PROTECTED,1652,4315,90.71
PROTECTED,154,442,9.29


### TASK 8

Is the range of tracks in the store reflective of their sales popularity?

finding the number of specific track_ids sold specific to a genre . compared to how many track_ids with that specific genre type is available.

a specific track_id may have been sold several times .nb for the store we just need one track_id for each song and this single track_id can be purcahsed several times. it doesnt operate like 1 track in inventory=1 purchase but rather 1 track in inventory = several purchases 

In [279]:

%%sql
WITH FINAL_TABLE AS (SELECT g.name,t.track_id,il.track_id as sold_id from track t

                     LEFT JOIN invoice_line il ON t.track_id=il.track_id 

                     LEFT JOIN genre g ON g.genre_id =t.genre_id)

    

SELECT  name, COUNT (DISTINCT track_id) as "number of tracks with genre type", COUNT(DISTINCT sold_id) " number of genre type track sold",ROUND(CAST(COUNT(DISTINCT track_id) AS FLOAT) * 100
    /
    (SELECT COUNT(*) FROM track),2) AS 'Inventory tracks % by genre',ROUND((CAST(COUNT(DISTINCT sold_id) AS FLOAT)/COUNT (DISTINCT track_id))*100,2) as percentage_sold

     FROM final_table GROUP BY 1 ORDER BY 2 DESC



 * sqlite:///chinook.db
Done.


name,number of tracks with genre type,number of genre type track sold,Inventory tracks % by genre,percentage_sold
Rock,1297,915,37.03,70.55
Latin,579,119,16.53,20.55
Metal,374,238,10.68,63.64
Alternative & Punk,332,176,9.48,53.01
Jazz,130,61,3.71,46.92
TV Shows,93,2,2.65,2.15
Blues,81,56,2.31,69.14
Classical,74,16,2.11,21.62
Drama,64,1,1.83,1.56
R&B/Soul,61,55,1.74,90.16


### Conclusion

In this project, we have analyzed a modified version of the Chinook database for finding the answers to our business questions. Below are the main results of this work (emphasized are the factors and suggestions for maximizing overall revenue):

From the 4 albums in consideration, the following 3 ones should be selected for purchasing and advertising in the USA, based on the genre popularity rating in this country: Red Tone (Punk), Slim Jim Bites (Blues), Meteor and the Girls (Pop).
Of the 3 sales support agents, Jane Peacock demonstrates the most effective sales performance, while Margaret Park - the least. The main contributing factor is the employed service; other factors are the number of clients and the employee's age.
Of all the countries, the USA represents the major part of the market, in terms of both the total sales and the number of customers.
Customers tend to buy individual tracks much more often than the whole albums. Hence, it's recommended to the Chinook store to purchase only the most popular tracks from each album from record companies.
The tracks of the artists present in the most playlists are all of Classical genre, which is not popular among customers. Purchasing them cannot be considered as a potentially profitable startegy for the Chinook store.
Almost half of all the unique tracks of the Chinook store were never bought. Possible solutions here may include a new advertising campaign, sales promotion, additional training and motivation of sales support agents.
The tracks of protected media types are much less popular than those of non-protected, both in terms of the store assortment and, especially, in terms of sales. Hence the store should avoid purchasing such tracks.